# Pytorch Tutorial

In [1]:
import torch
torch.cuda.is_available()

True

## 1. Scalars, Vectors, Matrices and Tensors

In [3]:
import torch
import numpy as np

# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])

# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2], [3, 4]])

# create a 3D tensor from a nested Python list
tensor3d_1 = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

# create a 3D tensor from NumPy array
ary3d = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
tensor3d_2 = torch.tensor(ary3d)  # Copies NumPy array
tensor3d_3 = torch.from_numpy(ary3d)  # Shares memory with NumPy array

In [5]:
print(tensor1d.dtype)

torch.int64


A 32-bit floating point number offers sufficient precision while consuming less memory and computational resources than a 64-bit floating point number

In [6]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

torch.float32


## 2. Common PyTorch Tensor Operations

### Shape

In [7]:
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(tensor2d)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [8]:
tensor2d.shape

torch.Size([2, 3])

### Reshape

In [11]:
tensor2d.reshape(3, 2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [12]:
# more common command for reshaping tensors in PyTorch
tensor2d.view(3, 2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

### Matrix multiplication

In [13]:
tensor2d.matmul(tensor2d.T)

tensor([[14, 32],
        [32, 77]])

In [14]:
tensor2d @ tensor2d.T

tensor([[14, 32],
        [32, 77]])

## 3. Automatic Differentiation Engine

- PyTorch's autograd system provides functions to compute gradients in dynamic computational graphs automatically
- A graph will be built internally by default if one of its terminal nodes has the ``requires_grad = True`` attribute

In [15]:
import torch.nn.functional as F
from torch.autograd import grad
 
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

In [16]:
z = x1 * w1 + b
z

tensor([2.4200], grad_fn=<AddBackward0>)

In [17]:
a = torch.sigmoid(z)
a

tensor([0.9183], grad_fn=<SigmoidBackward0>)

In [19]:
loss = F.binary_cross_entropy(a, y)
loss

tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)

In [20]:
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

In [21]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In practice, there is a more high level way to use it

In [22]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


## 4. Multilayer Neural Networks

In [23]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(in_features=num_inputs, out_features=30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(in_features=30, out_features=20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(in_features=20, out_features=num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [25]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


Checking the total number of trainable parameters

In [31]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [35]:
model.layers[0]

Linear(in_features=50, out_features=30, bias=True)

In [37]:
model.layers[0].weight

Parameter containing:
tensor([[-0.0512, -0.0311, -0.0197,  ...,  0.0770,  0.0722,  0.0340],
        [ 0.0374,  0.0895, -0.0624,  ...,  0.0832, -0.0621,  0.0986],
        [ 0.0419,  0.0835,  0.0275,  ...,  0.0602, -0.0392,  0.1369],
        ...,
        [ 0.0243,  0.0405, -0.0906,  ..., -0.0417, -0.0756, -0.0157],
        [-0.0227, -0.0251, -0.0920,  ..., -0.0022,  0.0853,  0.1236],
        [ 0.0679,  0.0744, -0.0698,  ..., -0.0960, -0.1409,  0.0967]],
       requires_grad=True)

In [38]:
model.layers[0].weight.shape

torch.Size([30, 50])

Doing ``model(x)`` automatically execute the forward pass of the model

In [39]:
torch.manual_seed(123)
X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.2121, -0.2315,  0.1578]], grad_fn=<AddmmBackward0>)


If we want to use network without training or backpropagation, we use ``torch.no_grad()``

In [41]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.2917, 0.2861, 0.4222]])


## 5. Setting up efficient data loaders

Dataloader is efficient for processing data during training or evaluation of neural networks.
- Automatically batches the data, allowing us to process multiple samples together
- Can shuffle data at the beginning of each epoch, which helps prevent model from memorizing the order of training examples and encourages better generalisation
- Allows for parallel data loading and processing
- Can apply various data transformations
- Makes it easy to loop over batches of data during training or evaluation

In [42]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])
 
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [43]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]
    

In [44]:
train_ds = ToyDataset(X=X_train, y=y_train)
test_ds = ToyDataset(X=X_test, y=y_test)

In [46]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(dataset=train_ds,
                          batch_size=2,
                          shuffle=True,
                          num_workers=0)

test_loader = DataLoader(dataset=test_ds,
                         batch_size=2,
                         shuffle=False,
                         num_workers=0)

- ``train_loader`` iterates over the training dataset. It visits each training example once.
- Since the last batch is not divisible by 2, we will drop the last batch in each epoch
- When ``num_workers > 0``, multiple worker processes are launched to load data in parallel, freeing the main process to focus on training your model and better utilizing your system's resources

In [47]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [48]:
train_loader = DataLoader(dataset=train_ds,
                          batch_size=2,
                          shuffle=True,
                          num_workers=0,
                          drop_last=True)

In [49]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


## 6. A Typical Training Loop

In [50]:
import torch.nn.functional as F
 
 
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
 
num_epochs = 3
 
for epoch in range(num_epochs): 
    
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
 
        logits = model(features)
        
        loss = F.cross_entropy(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")
 
    model.eval()
    # Optional model evaluation

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


Making predictions

In [51]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


To obtain the class membership probabilities,

In [52]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


Convert these values into class labels predictions by returning the index position of the highest value in each row

In [54]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


Count the number of correct prediction

In [55]:
torch.sum(predictions == y_train)

tensor(5)

To compute the prediction accuracy,

In [62]:
def compute_accuracy(model, dataloader):
 
    model = model.eval()
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        
        with torch.no_grad():
            logits = model(features)
        
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)
 
    return (correct / total_examples).item()

In [63]:
print(compute_accuracy(model, train_loader))

1.0


## 7. Saving and Loading Models

In [64]:
# Save
torch.save(model.state_dict(), "model.pth")

In [65]:
# Load
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

## 8. Computations on GPU devices

In [66]:
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


We will transfer these tensors onto a GPU and perform addition there. The output shows that the tensors reside on the first GPU.

In [67]:
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")
print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


All tensors must be on the same device or compuation will fail

## 9. Training on GPU

In [68]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
 
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
 
num_epochs = 3
 
for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
 
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels) # Loss function
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")
 
    model.eval()
    # Optional model evaluation
 

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00
